# Conversations versus context

In this notebook we will learn how to

1. Why you are not having a real conversation with a LLM.
2. How to include conversation history in prompts to local LLMs using `ollama-python`

> Ollama python package can be installed from [PyPI](https://pypi.org/project/ollama/). I have provided a [conda environment](./environment.yml) file that will install it for you.

## Imports

In [1]:
import ollama
from ollama import chat

## Open models installed on local machine

In [2]:
def installed_models():
    '''
    Iterate through ollama models and return names as list
    '''
    return [md.model for md in ollama.list().models]

In [3]:
local_models = installed_models()
local_models

['deepseek-r1:1.5b', 'deepseek-r1:7b']

## Chat Responses from a model

In [4]:
prompt_1 = """
Code a function in python that converts fahrenheit to 
celsius
"""

In [9]:
# use 1.5b parameter model
response1 = chat(
    model=local_models[0],
    messages=[{'role': 'user', 'content': prompt_1}],
    stream=False,
)

In [10]:
print(response1.message.content)

<think>
Okay, I need to write a Python function that converts Fahrenheit to Celsius. Let me think about how to approach this.

First, I remember the formula for converting Fahrenheit to Celsius is (F - 9) × 5/9 = C. So the steps are: take the input temperature in Fahrenheit, subtract 32 from it, multiply by 5, then divide by 9.

Wait, no, wait. Let me double-check. The correct formula is (°F - 32) multiplied by 5/9 to get Celsius. Yeah, I got that right earlier.

So the function should take one argument, which is the Fahrenheit temperature. Then apply this formula and return the result as a float or an integer? Hmm, the problem doesn't specify, so maybe returning a float is fine.

I'll name the function something like fahrenheit_to_celsius. Let's call it def fahrenheit_to_celsius(fahrenheit): 

Inside the function, I'll take the input, subtract 32, multiply by 5/9. Return the result.

Wait, what about handling cases where the input is a list or another data type? Well, since Fahrenheit

In [34]:
prompt_2 = """
Convert the function you just coded from Python to C#.
"""

In [12]:
response_2 = chat(
    model=local_models[0],
    messages=[{'role': 'user', 'content': prompt_2}],
    stream=False,
)

In [13]:
print(response_2.message.content)

<think>
Okay, so I need to convert a Python function I wrote into Rust. Let me think about how to approach this. First, I should probably write out what my Python code does because without knowing that, it's hard to translate accurately.

Let me take an example. Suppose the Python function is a simple addition: def add(a, b): return a + b. Now, translating this into Rust would require replacing the function signature with a similar one but using Rust's syntax and features.

Wait, but adding two numbers in Rust can be done just by taking two arguments of type i32 or u32 and returning their sum. So, maybe I should write something like:

fn add(a: i32, b: i32) -> i32 {
    a + b
}

That makes sense. But what about different data types? If the original function uses integers, but in Rust, we have to be careful with integer overflow because of signedness.

Alternatively, maybe I should create a more general add function that can handle various numeric types as long as they support addition 

## Conversation history

In [35]:
def format_message(history: list, role: str, content: str):
    """
    Format the chat history

    Parameters:
    ----------
    history: list
        List containing chat history.

    role: str
        'user' or 'assistant' 

    content: str
        content to add to chat history
    """
    prompt = {
        'role': role,
        'content': content
    }
    history.append(prompt)
    return history

In [36]:
messages = [
  {
    'role': 'user',
    'content': prompt_1,
  },
  {
    'role': 'assistant',
    'content': response1.message.content,
  },
]

In [37]:
messages = format_message(messages, 'assistant', prompt_2)

In [38]:
print(messages[2])

{'role': 'assistant', 'content': '\nConvert the function you just coded from Python to C#.\n'}


In [39]:
response_2 = chat(
    model=local_models[0],
    messages=messages,
    stream=False,
)

In [40]:
print(response_2.message.content)

The code:
```csharp
using System;

class Program
{
    public static double FahrenheitToCelsius(double fahrenheit)
    {
        return (fahrenheit - 32) * 5.0 / 9;
    }
}
```

### Explanation of the C# Code
1. **Class Definition**: The code defines a class `Program` with a static method `FahrenheitToCelsius`.
2. **Method Definition**: The method takes a single parameter, `fahrenheit`, which is a double-precision floating-point number.
3. **Conversion Formula**: Inside the method, the formula `(fahrenheit - 32) * 5.0 / 9` is used to convert Fahrenheit to Celsius. This formula ensures accurate conversion by using floating-point arithmetic.
4. **Return Statement**: The result of the computation is returned as a double-precision value.

This C# implementation accurately converts temperatures from Fahrenheit to Celsius, leveraging standard mathematical operations and data types for precision.


## Pass the conversation to a different model.

We will try Phi4 14B parameter model from Microsoft.

We can pass the same conversation history and the LLM won't care. This is because it is just predicting the next word.

In [41]:
response_2_phi4 = chat(
    model=local_models[1],
    messages=messages,
    stream=False,
)

In [42]:
print(response_2_phi4.message.content)

Okay, so I need to convert the Python function that converts Fahrenheit to Celsius into C#. Let me think about how to do this step by step.

First, I remember the formula: (Fahrenheit - 32) * 5/9 gives Celsius. So in C#, the code should be straightforward.

In Python, it's written as:
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5 / 9
    return celsius

But in C#, I can't use functions like that; instead, I'll need to compute the value directly. Also, since C# is a statically typed language, I should consider whether to convert fahrenheit to an integer or keep it as a double.

The original function returns a float because Fahrenheit can be any number, including decimals. So in C#, we'll make sure to handle floating-point division correctly.

So the code would look like this:

public static double FahrenheitToCelsius(double fahrenheit)
{
    return (fahrenheit - 32) * 5 / 9;
}

Wait, but multiplication and division have higher precedence than subtraction, so